In [ ]:
docker buildx build \
    --platform=linux/amd64 \
    --provenance=false \
    -f ./docker/Dockerfile \
    -t ${ECR_REPO_NAME}:v3 \
    --load \
    .


docker tag ${ECR_REPO_NAME}:v3 ${ECR_IMAGE_V3}
docker push ${ECR_IMAGE_V3}

aws sagemaker create-model \
    --model-name ${MODEL_NAME}-v3 \
    --execution-role-arn ${SAGEMAKER_ROLE} \
    --primary-container Image=${ECR_IMAGE_V3},ModelDataUrl=s3://${BUCKET_NAME}/${MODEL_ARTIFACT_KEY} \
    --region ${AWS_REGION}

In [ ]:
docker run --rm -it --platform linux/amd64 --entrypoint /bin/bash -p 8080:8080 -e MODEL_DIR='/tmp' sleep-predictor:v3


python -c "from api.app import app; print('Import Successful')"


uvicorn api.app:app --host 0.0.0.0 --port 8080

exit

In [ ]:
MODEL_NAME: sleep-disorder-svm-model-v1
SAGEMAKER_ROLE: arn:aws:iam::137568342316:role/SageMakerExecutionRole
ECR_IMAGE_V4: 137568342316.dkr.ecr.us-east-1.amazonaws.com/sleep-predictor:v4
BUCKET_NAME: sleep-disorder-mlops-bucket
MODEL_ARTIFACT_KEY: sagemaker-tuning-output/svm-tuning-251203-2103-002-c8ccdacc/output/model.tar.gz
AWS_REGION: us-east-1

aws sagemaker create-model \
    --model-name sleep-disorder-svm-model-v1-v4 \
    --execution-role-arn arn:aws:iam::137568342316:role/SageMakerExecutionRole \
    --primary-container Image=137568342316.dkr.ecr.us-east-1.amazonaws.com/sleep-predictor:v4,ModelDataUrl=s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/svm-tuning-251203-2103-002-c8ccdacc/output/model.tar.gz \
    --region us-east-1

aws iam get-role --role-name SageMakerExecutionRole --region us-east-1 --query 'Role.AssumeRolePolicyDocument'
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}




SyntaxError: invalid syntax (2655360442.py, line 1)

(mlops-env) zhengchengsheng@MacBook-Air-2 sleeping_disorder_prediction % docker run --rm -it --platform linux/amd64 --entrypoint /bin/bash -p 8080:8080 -e MODEL_DIR='/tmp' sleep-predictor:v3
root@e5a833c75685:/app# python -c "import pandas; print('✅ Pandas is installed!')"
bash: !': event not found
root@e5a833c75685:/app# python -c "from api.app import app; print('Import Successful')"
Import Successful
root@e5a833c75685:/app# uvicorn api.app:app --host 0.0.0.0 --port 8080
INFO:     Started server process [15]
INFO:     Waiting for application startup.
❌ 模型加载失败: [Errno 2] No such file or directory: 'model.joblib'
❌ 本地路径也加载失败: [Errno 2] No such file or directory: '../notebooks/best_model_extracted/model.joblib'
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)

(mlops-env) zhengchengsheng@MacBook-Air-2 sleeping_disorder_prediction % tree -L 3
.
├── README.md
├── adjustment_for_project.ipynb
├── api
│   ├── app.py
│   └── request_schema.json
├── config.yaml
├── data
│   └── Sleep_health_and_lifestyle_dataset.csv
├── docker
│   └── Dockerfile
├── docs
│   ├── architecture.md
│   ├── deploy_manager.py
│   ├── model_metadata.json
│   └── temp_operation_record.md
├── notebooks
│   ├── 01_sagemaker_orchestration.ipynb
│   ├── best_model_extracted
│   │   ├── label_encoder.joblib
│   │   └── model.joblib
│   ├── debug_logger_test.py
│   └── model.tar.gz
├── requirements.txt
├── s3_log_uploader.py
├── scripts
│   ├── debug_sagemaker_perms.py
│   └── upload_raw_data.py
├── setup_skeleton.py
├── src
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-39.pyc
│   │   ├── data_processor.cpython-39.pyc
│   │   └── train.cpython-39.pyc
│   ├── data_processor.py
│   ├── requirements.txt
│   └── train.py
└── tests
    ├── __init__.py
    ├── __pycache__
    │   ├── __init__.cpython-39.pyc
    │   ├── test_data.cpython-39-pytest-8.4.2.pyc
    │   └── test_training.cpython-39-pytest-8.4.2.pyc
    ├── test_data.py
    └── test_training.py

12 directories, 34 files

import pandas as pd
import joblib
import os
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# 1. 定义请求数据格式 (基于 request_schema.json)
class SleepInput(BaseModel):
    gender: str
    age: int
    occupation: str
    sleep_duration: float
    quality_of_sleep: int
    physical_activity_level: int
    stress_level: int
    bmi_category: str
    blood_pressure: str
    heart_rate: int
    daily_steps: int

# 2. 初始化 FastAPI 应用
app = FastAPI(title="Sleep Disorder Prediction API")

# 全局变量加载模型
model_pipeline = None
label_encoder = None

@app.on_event("startup")
def load_artifacts():
    global model_pipeline, label_encoder
    # 假设在 Docker 中模型位于 /opt/ml/model 目录
    # 在本地测试时，请确保这两个文件在当前目录下或修改路径
    model_path = os.getenv("MODEL_PATH", "model.joblib")
    le_path = os.getenv("LE_PATH", "label_encoder.joblib")
    
    try:
        model_pipeline = joblib.load(model_path)
        label_encoder = joblib.load(le_path)
        print("✅ 模型加载成功")
    except Exception as e:
        print(f"❌ 模型加载失败: {e}")
        # 本地调试时的备用路径 (根据您的 notebook 结构)
        try:
            base_path = "../notebooks/best_model_extracted/"
            model_pipeline = joblib.load(os.path.join(base_path, "model.joblib"))
            label_encoder = joblib.load(os.path.join(base_path, "label_encoder.joblib"))
            print("✅ 模型加载成功 (本地调试路径)")
        except Exception as e2:
            print(f"❌ 本地路径也加载失败: {e2}")

@app.get("/ping")
def health_check():
    """AWS SageMaker 需要的健康检查接口"""
    if model_pipeline is not None:
        return {"status": "Healthy"}
    raise HTTPException(status_code=500, detail="Model not loaded")

@app.post("/invocations")
async def predict(input_data: SleepInput):
    """AWS SageMaker 需要的推理接口 (路径必须是 /invocations)"""
    if not model_pipeline or not label_encoder:
        raise HTTPException(status_code=500, detail="Model not initialized")
    
    try:
        # 1. 转换为 DataFrame (这是 Pipeline 预期的输入格式)
        # 注意：这里需要手动处理列名，使其与训练时的标准化列名一致
        data_dict = input_data.dict()
        df = pd.DataFrame([data_dict])
        
        # 2. 进行预测
        pred_encoded = model_pipeline.predict(df)
        
        # 3. 解码结果 (0 -> Insomnia)
        pred_label = label_encoder.inverse_transform(pred_encoded)[0]
        
        return {"prediction": pred_label}
        
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# 本地测试启动命令: uvicorn api.app:app --reload





